# 📊 Slowly Changing Dimension Type 2 (SCD2) Demo

1. Load the initial dimension table (`customers.csv`).
2. Load new incoming data (`staging_customer.csv`).
3. Apply SCD2 ETL logic.
4. Compare before vs. after tables.


In [3]:
import pandas as pd
from datetime import datetime

# Load initial dimension and staging
dim = pd.read_csv("data/customers.csv")
stg = pd.read_csv("data/staging_customer.csv")

print("Initial Dimension Table:")
display(dim)

print("Staging Data (new load):")
display(stg)

Initial Dimension Table:


,customer_id,name,city,valid_from,valid_to,is_current
0,101,Alice,Manila,2020-01-01,2021-06-30,0
1,101,Alice,Cebu,2021-07-01,NaN,1
2,102,Bob,Cebu,2020-05-01,NaN,1
3,103,Charlie,Davao,2021-03-10,2022-02-28,0
4,103,Charlie,Cagayan de Oro,2022-03-01,NaN,1
5,104,Diana,Baguio,2019-11-15,NaN,1
6,105,Edward,Makati,2020-04-20,2023-04-30,0
7,105,Edward,Taguig,2023-05-01,NaN,1
8,106,Fatima,Quezon City,2021-08-12,NaN,1
9,107,George,Cebu,2021-01-01,2022-08-31,0


Staging Data (new load):


,customer_id,name,city
0,101,Alice,Davao
1,102,Bob,Cebu
2,103,Charlie,Cagayan de Oro
3,104,Diana,Baguio
4,105,Edward,Taguig
5,106,Fatima,Pasay
6,107,George,Davao
7,108,Hannah,Iloilo
8,109,Ivan,Mandaluyong
9,110,Jenny,Manila


## 🔄 Apply SCD2 Logic
If the attribute (e.g., city) changes → we close the old record and add a new one.

If the customer is new → we insert them.

If nothing changes → we leave the record as-is.

In [4]:
today = datetime.today().date()
updates = []
inserts = []

for _, row in stg.iterrows():
    current = dim[(dim["customer_id"] == row["customer_id"]) & (dim["is_current"] == 1)]

    if current.empty:
        # New customer
        inserts.append({
            "customer_id": row["customer_id"],
            "name": row["name"],
            "city": row["city"],
            "valid_from": today,
            "valid_to": None,
            "is_current": 1
        })
    else:
        if current.iloc[0]["city"] != row["city"]:
            # Close old record
            dim.loc[current.index, ["valid_to", "is_current"]] = [today, 0]
            # Insert new version
            updates.append({
                "customer_id": row["customer_id"],
                "name": row["name"],
                "city": row["city"],
                "valid_from": today,
                "valid_to": None,
                "is_current": 1
            })
        # else --> No change

if updates:
    dim = pd.concat([dim, pd.DataFrame(updates)], ignore_index=True)
if inserts:
    dim = pd.concat([dim, pd.DataFrame(inserts)], ignore_index=True)

print(f"Inserted: {len(inserts)}, Updated: {len(updates)}, Total now: {len(dim)} rows")

Inserted: 3, Updated: 5, Total now: 31 rows


## ✅ Updated Dimension Table

In [5]:
display(dim.sort_values(["customer_id", "valid_from"]))

,customer_id,name,city,valid_from,valid_to,is_current
23,101,Alice,Davao,2025-09-25,None,1
0,101,Alice,Manila,2020-01-01,2021-06-30,0
1,101,Alice,Cebu,2021-07-01,2025-09-25,0
2,102,Bob,Cebu,2020-05-01,NaN,1
3,103,Charlie,Davao,2021-03-10,2022-02-28,0
4,103,Charlie,Cagayan de Oro,2022-03-01,NaN,1
5,104,Diana,Baguio,2019-11-15,NaN,1
6,105,Edward,Makati,2020-04-20,2023-04-30,0
7,105,Edward,Taguig,2023-05-01,NaN,1
24,106,Fatima,Pasay,2025-09-25,None,1
